In [56]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path
import os, sys
import numpy as np
from utils import losses
import torch
from utils.utils import *

In [57]:
base_path = "./pro_data/PSPNet_VOC/train/"
files = os.listdir(base_path + "output")
cross_entropy_loss_func = losses.CrossEntropyLoss2d(reduction="none")

In [52]:
true_losses = None
for file in files:
    output = np_read(base_path + f"output/{file}")
    target = np_read(base_path + f"target/{file}")
    losses = cross_entropy_loss_func(torch.from_numpy(output), torch.from_numpy(target))
    losses = losses.numpy()
    np_write(losses, base_path + f"loss/{file}")

In [37]:
np_write(true_losses, base_path + "image_true_losses.npy")

In [55]:
outputs = None
for file in files[:30]:
    output = np_read(base_path + f"output/{file}")
    if outputs is None:
        outputs = output
    else:
        outputs = np.concatenate((outputs, output), axis=0)
print(outputs.mean(). outputs.std())

AttributeError: 'numpy.float32' object has no attribute 'outputs'

In [45]:
print(outputs.mean(axis=1). outputs.std(axis=1))

(8, 480, 480)

In [40]:
z = torch.zeros((3,4,5))

In [41]:
z[0].shape

torch.Size([4, 5])